In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
import sqlite3

def retrieve(word):
    '''
    queries recipe database for titles including a specified word
    Parameters:
        word (str): keyword for parsing the recipe data
    Returns:
        pandas DataFrame with relevant recipes
    '''
    
    with sqlite3.connect("recipes1M.db") as conn:
        cmd = \
        f"""
        SELECT R.title
        FROM recipes R
        WHERE R.title LIKE "%{word}%"
        """
        df = pd.read_sql_query(cmd, conn)
    
    return df

In [3]:
df = retrieve("salmon")
df

,title
0,Salmon & Salad a La SPORTZ
1,Curried Pumpkin and Smoked Salmon Soup
2,Grilled Rosemary Salmon Spedini
3,Garlic and Dill Salmon
4,Spicy Grilled Orange Salmon
...,...
10788,Creamy Smoked Salmon and Dill Frittata
10789,Salmon and Spaghetti Casserole
10790,Salmon Pot Pie
10791,Pesto-Crusted Salmon Fillet With Citrus-Soy Sauce


In [4]:
# split up the data into the text and the ideal predicted label

def give_input_split(title):
    ''' gives the string but without the last space and the text following it'''
    split_text = title.rsplit(" ", maxsplit = 1)
    return split_text[0]

def give_output_split(title):
    ''' gives the word following the last space of the text'''
    split_text = title.rsplit(" ", maxsplit = 1)
    if(len(split_text) < 2):
        return ""
    return split_text[1]

In [5]:
# create new columns based on the previous functions
df["input"] = df["title"].apply(give_input_split)
df["predict"] = df["title"].apply(give_output_split)

In [6]:
data = tf.data.Dataset.from_tensor_slices((df["input"], df["predict"]))
print(data)

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>


In [7]:
for headline, category in data.take(5): # check the first few entries
    print(headline)
    print(category)
    print("")

tf.Tensor(b'Salmon & Salad a La', shape=(), dtype=string)
tf.Tensor(b'SPORTZ', shape=(), dtype=string)

tf.Tensor(b'Curried Pumpkin and Smoked Salmon', shape=(), dtype=string)
tf.Tensor(b'Soup', shape=(), dtype=string)

tf.Tensor(b'Grilled Rosemary Salmon', shape=(), dtype=string)
tf.Tensor(b'Spedini', shape=(), dtype=string)

tf.Tensor(b'Garlic and Dill', shape=(), dtype=string)
tf.Tensor(b'Salmon', shape=(), dtype=string)

tf.Tensor(b'Spicy Grilled Orange', shape=(), dtype=string)
tf.Tensor(b'Salmon', shape=(), dtype=string)



In [8]:
# separate data into train-test-validate

data = data.shuffle(buffer_size = len(data))
train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))

train = data.take(train_size)
val   = data.skip(train_size).take(val_size)
test  = data.skip(train_size + val_size)
len(train), len(val), len(test)

(7555, 1079, 2159)

In [10]:
def standardize(input):
    lwer = tf.strings.lower(input)
    punc = tf.strings.regex_replace(lwer, '[%s]' % re.escape(string.punctuation), '')
    return punc 

In [11]:
# set up two functions, one to vectorize input and one to vectorize the predicted values

max_tokens = 5000
sequence_length = 15

vectorize_input = TextVectorization(
    standardize = standardize,
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = sequence_length) 

vectorize_predict = TextVectorization(
    standardize = standardize,
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = 1)

In [12]:
recipes = train.map(lambda x, y: x) # collapse the columns into a single one
vectorize_input.adapt(recipes) # tell the vectorizers what words exist
vectorize_predict.adapt(recipes)

In [13]:
list(recipes.take(1))

[<tf.Tensor: shape=(), dtype=string, numpy=b'Smoked Salmon Tea'>]

In [14]:
list(train.take(1))

[(<tf.Tensor: shape=(), dtype=string, numpy=b'Peppered Salmon W/ Arugula (Rocket) and Yogurt Dressed'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'Potatoes'>)]

In [15]:
def vectorize_recipe(text, label):
    text = tf.expand_dims(text, -1)
    label = tf.expand_dims(label, -1)
    return vectorize_input(text), vectorize_predict(label)

train_vec = train.map(vectorize_recipe)
val_vec   = val.map(vectorize_recipe)
test_vec  = test.map(vectorize_recipe)

In [16]:
list(val_vec.take(5))

[(<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
  array([[53,  4,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
        dtype=int64)>,
  <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[185]], dtype=int64)>),
 (<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
  array([[   2,    4, 2095,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0]], dtype=int64)>,
  <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[250]], dtype=int64)>),
 (<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
  array([[ 25,   7,   2, 131,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0]], dtype=int64)>,
  <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[1]], dtype=int64)>),
 (<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
  array([[42,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
        dtype=int64)>,
  <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[12]], dtype=int64)>),
 (<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
  array([[ 48,  2

In [17]:
train_vec # shows the shape of our tensors

<MapDataset shapes: ((None, 15), (None, 1)), types: (tf.int64, tf.int64)>

In [23]:
vocabulary1 = vectorize_input.get_vocabulary() # collect all the words used
vocabulary2 = vectorize_predict.get_vocabulary()

In [47]:
def num_to_str(vec, vocab1, vocab2):
    ''' converts numeric recipe to original title '''
    arr1 = vec[0].numpy() # get list of numbers that make up recipe input
    arr1 = arr1[0] # reduce dimension by one
    arr2 = vec[1].numpy() # get number that makes up desired recipe output
    val2 = arr2[0][0] # reduce dimensions appropriately
    title = ""
    for num in arr1: # for all numbers, we need to add on the right word
        if num != 0: # don't want to add on spaces
            title += vocab1[num] + " "
    title += vocab2[val2]
    return title

In [49]:
sample = list(train_vec.take(10))

for item in sample:
    out = num_to_str(item, vocabulary1, vocabulary2)
    print(out)

cedarplanked salmon with washington state merlot reduction and garlic spinach
delicious pistachio encrusted salmon
salmon steamed over orangebasil tomato sauce
smoked salmon and horseradish cheesecake
salmon [UNK]
grilled salmon caesar salad
super herbed sauteed salmon with creamy leeks and bacon
panseared salmon puttanesca
salmon asparagus pie
salmon with rice [UNK]


In [ ]:
# does not work

def split_input_target(title):
    input_text = title[:-1]
    target_text = title[1:]
    
    return input_text, target_text

dataset_targeted = train_vec.map(split_input_target)

print(dataset_targeted)

This code was David's attempt to transpose the data structure for the model. It was not successful.

In [54]:
tempTensor = list(train_vec.take(1))[0][0]
tempTensor

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 25, 740,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [55]:
tf.transpose(tempTensor)

<tf.Tensor: shape=(15, 1), dtype=int64, numpy=
array([[ 25],
       [740],
       [  4],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0]], dtype=int64)>

In [56]:
test = train_vec.map(lambda x, y: tf.transpose(a = x))
list(test.take(1))

[<tf.Tensor: shape=(15, 1), dtype=int64, numpy=
 array([[ 107],
        [   2],
        [   3],
        [2690],
        [  19],
        [   4],
        [  75],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0],
        [   0]], dtype=int64)>]

In [57]:
train_vec = test
#train_vec = train_vec.map(lambda x, y: tf.transpose(a = x))
val_vec   = val_vec.map(lambda x, y: tf.transpose(a = x))
test_vec  = test_vec.map(lambda x, y: tf.transpose(a = x))

In [58]:
list(val_vec.take(1))

[<tf.Tensor: shape=(15, 1), dtype=int64, numpy=
 array([[ 2],
        [ 4],
        [43],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]], dtype=int64)>]

This is the end of the transposing code.

In [50]:
VOCABULARY_SIZE = max_tokens +1

In [59]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))

    model.add(tf.keras.layers.LSTM(
        units=rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer=tf.keras.initializers.GlorotNormal()
    ))

    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

model = build_model(
  vocab_size=max_tokens + 1,
  embedding_dim=256,
  rnn_units=1024,
  batch_size= sequence_length
)

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (15, None, 256)           1280256   
_________________________________________________________________
lstm_1 (LSTM)                (15, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (15, None, 5001)          5126025   
Total params: 11,653,257
Trainable params: 11,653,257
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [61]:
history = model.fit(train_vec, epochs = 10, validation_data = val_vec)

Epoch 1/10


ValueError: in user code:

    C:\Users\david\Documents\anaconda3\envs\PIC16B\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\david\Documents\anaconda3\envs\PIC16B\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\david\Documents\anaconda3\envs\PIC16B\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\david\Documents\anaconda3\envs\PIC16B\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\david\Documents\anaconda3\envs\PIC16B\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs

Error Text:
InvalidArgumentError:  Input to reshape is a tensor with 15 values, but the requested shape has 225
	 [[node gradient_tape/sequential/embedding/embedding_lookup/Reshape_1 (defined at <ipython-input-53-eddd9874a904>:1) ]] [Op:__inference_train_function_49112]

Function call stack:
train_function

The main difference between our code and the fitted code is that our tensors are shaped as tuples of (None, 15) and the ideal input is supposed to be (15, None).

Then if I run the transposing code, the error looks like a huge mess but the last line reads: 

ValueError: No gradients provided for any variable: ['embedding_1/embeddings:0', 'lstm_1/lstm_cell_1/kernel:0', 'lstm_1/lstm_cell_1/recurrent_kernel:0', 'lstm_1/lstm_cell_1/bias:0', 'dense_1/kernel:0', 'dense_1/bias:0'].

So far I have not been able to figure out what's going on.

In [62]:
tf.trainable_variables()

AttributeError: module 'tensorflow' has no attribute 'trainable_variables'